<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Sample notebook to build custom images from the lake user teamspace.

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)
2. [Running Container using the profile](#Running-container)


### Configuration

In [1]:
profile_name='simple_image'
image_name = 'custom_image2'
folder_name = 'simple'

### Build Image

Lets see how  orbit build image works...

In [2]:
!orbit build image --help

Usage: orbit build image [OPTIONS]

  Build and Deploy a new Docker image into ECR.

Options:
  -e, --env TEXT        Orbit Environment.  [required]
  -d, --dir TEXT        Dockerfile directory.  [required]
  -n, --name TEXT       Image name.  [required]
  -s, --script TEXT     Build script to run before the image build.
  -t, --team TEXT       One or more Teams to deploy the image to (can de
                        declared multiple times).

  --build-arg TEXT      One or more --build-arg parameters to pass to the
                        Docker build command.

  --debug / --no-debug  Enable detailed logging.  [default: False]
  --help                Show this message and exit.


get our orbit env and team names

In [3]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

('dev-env', 'lake-user')

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [4]:
repository_name = (f"orbit-{env_name}/users/{image_name}")
repository_name

'orbit-dev-env-users-custom_image2'

In [5]:
!aws ecr delete-repository --repository-name $repository_name --force

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:495869084367:repository/orbit-dev-env-users-custom_image2",
        "registryId": "495869084367",
        "repositoryName": "orbit-dev-env-users-custom_image2",
        "repositoryUri": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-dev-env-users-custom_image2",
        "createdAt": 1614284484.0,
        "imageTagMutability": "MUTABLE"
    }
}


In [6]:
%cd ~/shared/samples/notebooks/I-Image/$folder_name

/efs/shared/samples/notebooks/I-Images/simple


adding a file to our docker as an example

In [7]:
%%writefile my-custom-image-file.txt
just a text 

Overwriting my-custom-image-file.txt


In [8]:
pwd = %pwd
pwd

'/efs/shared/samples/notebooks/I-Images/simple'

Now lets run the command

In [9]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name
output

CPU times: user 476 µs, sys: 12.3 ms, total: 12.8 ms
Wall time: 5min 12s


['',
 'Deploying Docker Image |\x1b                   \x1b|   0% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b▏                  \x1b|   1% \x1b',
 'Deploying Docker Image |\x1b▌                  \x1b|   3% \x1b',
 'Deploying Docker Image |\x1b▉                  \x1b|   5% \x1b',
 'Deploying Docker Image |\x1b███▊               \x1b|  20% \x1b',
 'Deploying Docker Image |\x1b█████▏             \x1b|  27% \x1b',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 'Deploying Docker Image |\x1b███████████████████\x1b| 100% \x1b',
 '                

Lets get the image address from the output of the previous command

In [10]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-dev-env-users-custom_image2


In [11]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

orbit-dev-env-users-custom_image2
[{'imageDigest': 'sha256:e228607390e803db2fa88c1fd86d611e30c023b8f09a265d1f9a261551190c01', 'imageTag': 'latest'}]


### Building the profile for the Image

In [12]:
import json
profile = {
        "display_name": profile_name,
        "description": "Use simple custom image",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 2,
            "cpu_limit": 2,
            "mem_guarantee": "1G",
            "mem_limit": "1G"
        }
}

with open("profile.json", 'w') as f:
    json.dump(profile, f)


In [13]:
!cat profile.json

{"display_name": "simple_image", "slug": "nano", "description": "Use simple custom image", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-dev-env-users-custom_image2", "cpu_guarantee": 2, "cpu_limit": 2, "mem_guarantee": "1G", "mem_limit": "1G"}}

In [14]:
!orbit build profile --debug --env $env_name --team $team_name profile.json

[2021-02-25 21:22:06,967][__main__.py  :331] env: dev-env
[2021-02-25 21:22:06,967][__main__.py  :332] team: lake-user
[2021-02-25 21:22:06,967][__main__.py  :334] profile: {"display_name": "simple_image", "slug": "nano", "description": "Use simple custom image", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-dev-env-users-custom_image2", "cpu_guarantee": 2, "cpu_limit": 2, "mem_guarantee": "1G", "mem_limit": "1G"}}
[2021-02-25 21:22:06,967][__main__.py  :335] debug: True
[2021-02-25 21:22:06,967][messages.py  :133] Progress bar: 1%
[ Info ] Retrieving existing profiles
[2021-02-25 21:22:07,363][image.py     : 38] Trying to read profiles from SSM parameter (/orbit/dev-env/teams/lake-user/user/profiles).
[2021-02-25 21:22:07,507][image.py     :127] Existing user profiles for team lake-user: [{'description': 'Use simple custom image', 'display_name': 'simple_image', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'image': '495869084367.

### Running container 

Lets run a container using the profile and image we created

In [15]:
import json
run = {
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "profile": profile_name
      },
      "tasks":  [{
          "notebookName": "test-image.ipynb",
          "sourcePath": pwd,
          "targetPath": f"/efs/shared/regression/notebooks/I-Image/{folder_name}",
          "params": {
          }
        }]
 }

with open("run.json", 'w') as f:
    json.dump(run, f)


In [17]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json


INFO:root:using profile simple_image
INFO:root:Waiting for 1 tasks [{'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-fargate-runner-dq2r5', 'NodeType': 'fargate', 'tasks': [{'notebookName': 'test-image.ipynb', 'sourcePath': '/efs/shared/samples/notebooks/I-Images/simple', 'targetPath': '/efs/shared/regression/notebooks/I-Images/simple', 'params': {}, 'ExecutionType': 'ecs'}]}]
INFO:root:Watching task: 'orbit-lake-user-fargate-runner-dq2r5'
INFO:root:Running: 1 Completed: 0 Errored: 0
INFO:root:waiting for [{'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-fargate-runner-dq2r5', 'NodeType': 'fargate', 'tasks': [{'notebookName': 'test-image.ipynb', 'sourcePath': '/efs/shared/samples/notebooks/I-Images/simple', 'targetPath': '/efs/shared/regression/notebooks/I-Images/simple', 'params': {}, 'ExecutionType': 'ecs'}]}]
INFO:root:Task {'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-fargate-runner-dq2r5', 'NodeType': 'fargate', 'tasks': [{'notebookName': 'test-image.ipynb'